In [ ]:
# import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import cv2
import math
import torch
import json
import matplotlib
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np
import matplotlib.colorbar as colorbar
import pickle
import requests
from io import BytesIO
from PIL import Image
import numpy as np
from maskrcnn_benchmark.utils import cv2_util
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

from moviepy.editor import VideoFileClip

# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20*1.5, 12*1.5

from maskrcnn_benchmark.config import cfg
from predictor import COCODemo

In [ ]:
def load(img_path):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    pil_image = Image.open(img_path).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def load_video(video_path,frame_id=[0]):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    video_clip = VideoFileClip(video_path)
    if frame_id is None:
        nframes = int(video_clip.fps * video_clip.duration)
        frame_id = range(nframes)
    images = []
    for i in frame_id:
        image = video_clip.get_frame(i/video_clip.fps)
        # convert to BGR format
        image = np.array(image)[:, :, [2, 1, 0]]
        images += [image]
    video_clip.close()
    
    return images

def imshow(img):
    plt.gca().invert_yaxis()
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")

In [ ]:
# image directory
base = "../tools/datasets/bee/validation/"

# json file (just needs image names)
base_val = "../tools/datasets/bee/annotations/validation.json"

with open(base_val) as f:
    data_an = json.load(f)

test_files = []
for p in data_an['images'][:]:
    test_files.append(str(p['file_name']))

test_images = []
for file in test_files:
    test_images.append(load(base + file))



In [ ]:
# stores predictions
predictions = []

# stores images
predicted_images = []

In [ ]:
# get predictions from model
import matplotlib.gridspec as gridspec
from itertools import chain


model_weight = '/home/bsb2144/directpose/tools/training_dir/25/c_new/p2_standard/1/fcos_kps_ms_training_R_50_FPN_1x_bee/model_final.pth' 
config_file = "/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml"

cfg.merge_from_file(config_file)
cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                     'DATATYPE', 'bee', \
                     'MODEL.WEIGHT', model_weight,\
                     'DATASETS.TEST', "('bee_val_cocostyle',)",\
                     'DATASETS.TRAIN', "('bee_train_cocostyle', )",\
                    ])

coco_demo = COCODemo(
            cfg,
            min_image_size=800,
            confidence_threshold=0.20,
)
for idx, image in enumerate(test_images):
    predicted_image, prediction = coco_demo.run_on_opencv_image(image, include_predictions=True)
    predicted_images.append(predicted_image)
    predictions.append(prediction)
    

In [ ]:
# save predictions to json file
out_dir = "predictions.json" 
with open(out_dir, 'wb') as f:
    pickle.dump(predictions, f)

In [ ]:
# save predicted images to video
save_file = 'prediction_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

dd1,dd2 = (predicted_images[0].shape[0]),(predicted_images[0].shape[1])
out = cv2.VideoWriter(save_file, fourcc, 1.0, (dd2,dd1))

for image in predicted_images:
    out.write(image)
out.release()